In [ ]:
import gym
from gym import spaces
import numpy as np
import random
from collections import defaultdict

# ─── 1) TicTacToe Gym Environment ───────────────────────────────────────────────
class TicTacToeEnv(gym.Env):
    metadata = {'render.modes': ['human']}
    def __init__(self):
        super().__init__()
        # 0=empty, 1=X (agent), 2=O (opponent)
        self.observation_space = spaces.Discrete(3**9)
        self.action_space      = spaces.Discrete(9)
        self.reset()

    def reset(self):
        # create empty board
        self.board = np.zeros(9, dtype=int)
        self.done  = False
        return self._encode()

    def _encode(self):
        """Convert board[0..8] in {0,1,2} to single int in [0,3^9)."""
        code = 0
        for i, v in enumerate(self.board):
            code += (3**i) * int(v)
        return code

    def _decode(self, code):
        """Convert int code back to board array."""
        b = np.zeros(9, dtype=int)
        for i in range(9):
            b[i] = code % 3
            code //= 3
        return b

    def step(self, action):
        # invalid move
        if self.done or self.board[action] != 0:
            return self._encode(), -10.0, True, {}

        # agent move
        self.board[action] = 1
        if self._is_winner(1):
            self.done = True
            return self._encode(), +1.0, True, {}
        if 0 not in self.board:
            self.done = True
            return self._encode(), 0.0, True, {}  # draw

        # opponent random move
        empties = np.where(self.board == 0)[0]
        opp_act = random.choice(empties)
        self.board[opp_act] = 2
        if self._is_winner(2):
            self.done = True
            return self._encode(), -1.0, True, {}
        if 0 not in self.board:
            self.done = True
            return self._encode(), 0.0, True, {}

        # game continues
        return self._encode(), 0.0, False, {}

    def _is_winner(self, p):
        B = self.board.reshape(3,3)
        # rows, cols, diags
        win_states = [B[i,:] for i in range(3)] + \
                     [B[:,j] for j in range(3)] + \
                     [B.diagonal(), np.fliplr(B).diagonal()]
        return any((line == p).all() for line in win_states)

    def render(self, mode='human'):
        chars = ['.', 'X', 'O']
        B = [chars[v] for v in self.board]
        for i in range(3):
            print(' '.join(B[3*i:3*i+3]))
        print()

# ─── 2) Q‐Learning Agent ─────────────────────────────────────────────────────────
def encode_state(board):
    # already encoded by env, so we can reuse
    return board

def choose_action(Q, state, eps):
    if random.random() < eps:
        # random among legal moves
        b = env._decode(state)
        legal = [i for i,v in enumerate(b) if v==0]
        return random.choice(legal)
    # otherwise greedy among legal
    qvals = Q[state]
    b = env._decode(state)
    legal = [i for i,v in enumerate(b) if v==0]
    # pick legal action with highest Q
    best = max(legal, key=lambda a: qvals[a])
    return best

# ─── 3) Training Loop ────────────────────────────────────────────────────────────
env = TicTacToeEnv()

# hyperparameters
alpha   = 0.1
gamma   = 0.9
epsilon = 0.2
episodes = 50_000

# Q-table: default 0 for unseen states
Q = np.zeros((3**9, 9))

# stats
wins = draws = losses = 0

for ep in range(1, episodes+1):
    state = env.reset()
    done  = False

    while not done:
        action = choose_action(Q, state, epsilon)
        next_state, reward, done, _ = env.step(action)

        # compute max_a' Q(next_state,a')
        # but restrict to legal actions
        b_next = env._decode(next_state)
        legal_next = [i for i,v in enumerate(b_next) if v==0]
        if legal_next:
            max_q_next = max(Q[next_state, a] for a in legal_next)
        else:
            max_q_next = 0.0

        # Q-learning update
        Q[state, action] += alpha * (reward + gamma * max_q_next - Q[state, action])

        state = next_state

    # track stats
    if reward > 0:
        wins += 1
    elif reward < 0:
        losses += 1
    else:
        draws += 1

    # decay ε
    if ep % 5000 == 0:
        epsilon = max(0.05, epsilon * 0.9)

    # periodic report
    if ep % 5000 == 0:
        total = wins + draws + losses
        print(f"Episode {ep}: W/D/L = {wins}/{draws}/{losses}  "
              f"({wins/total:.2f}/{draws/total:.2f}/{losses/total:.2f})")
        wins = draws = losses = 0

# ─── 4) Test vs Random Opponent ─────────────────────────────────────────────────
test_eps = 0.0
test_games = 5000
wins = draws = losses = 0

for _ in range(test_games):
    state = env.reset()
    done  = False
    while not done:
        action = choose_action(Q, state, test_eps)
        state, reward, done, _ = env.step(action)
    if reward>0: wins +=1
    elif reward<0: losses +=1
    else: draws +=1

print(f"\nAfter training: W/D/L = {wins}/{draws}/{losses} "
      f"({wins/test_games:.2f}/{draws/test_games:.2f}/{losses/test_games:.2f})")


Episode 5000: W/D/L = 4095/356/549  (0.82/0.07/0.11)
Episode 10000: W/D/L = 4351/293/356  (0.87/0.06/0.07)
Episode 15000: W/D/L = 4404/309/287  (0.88/0.06/0.06)
Episode 20000: W/D/L = 4517/242/241  (0.90/0.05/0.05)
Episode 25000: W/D/L = 4632/193/175  (0.93/0.04/0.04)
Episode 30000: W/D/L = 4687/173/140  (0.94/0.03/0.03)
Episode 35000: W/D/L = 4701/177/122  (0.94/0.04/0.02)
Episode 40000: W/D/L = 4745/162/93  (0.95/0.03/0.02)
Episode 45000: W/D/L = 4771/138/91  (0.95/0.03/0.02)
Episode 50000: W/D/L = 4779/142/79  (0.96/0.03/0.02)

After training: W/D/L = 4963/37/0 (0.99/0.01/0.00)
